In [5]:
# Import Library
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
from pandas_datareader import data as web
import os
import string
import requests
import nltk
from nltk import word_tokenize
from nltk.corpus import PlaintextCorpusReader
from tqdm import tqdm
import time

In [74]:
# Stock Price
def stock(stock_type, start, end):
    sp500stock = web.get_data_yahoo([stock_type], start,end)
    return sp500stock
    
stock_type = '^GSPC'
# start= datetime(2008,7,1)
# end = datetime(2008,9,1)

# start= datetime(2008,9,1)
# end = datetime(2010,9,1)

# start= datetime(2010,9,1)
# end = datetime(2012,9,1)

# start= datetime(2012,9,1)
# end = datetime(2014,9,1)


start= datetime(2014,9,1)
end = datetime(2016,10,1)

stock_data = stock(stock_type, start, end)

In [75]:
open_date = stock_data.index
len(open_date)

526

## Web Scraping from Wall Street Journal

In [76]:
#Scraping news from the the corresponding date we found
date_loop_list = []
for i in range(len(open_date)):
    d1 = open_date[i]
    date_loop_list.append(d1.date())
print(len(date_loop_list))
# print(date_loop_list)

526


In [77]:
# Web Scraping from Wall Street Journal
def news(dateloop):
#     date_list = [base - datetime.timedelta(days=x) for x in range(daterange)]
#     date_loop_list = []
#     for i in date_list:
#         dt = datetime.date.strftime(i, '%Y-%m-%d')
#         date_loop_list.append(dt)
    
    All_News=[]
    for date in dateloop:
        date = pd.to_numeric(str(date).replace('-',''))
        url = 'http://www.wsj.com/news/archive/'+ str(date)
        page = requests.get(url)    
        result = BeautifulSoup(page.content,'lxml')
        UL = result.find_all('article')
        everyday_results = []
        Headlines_Day = []
        Abstracts_Day = []
        t = 0
        while len(UL)==0 or t == 20:
            url = 'http://www.wsj.com/news/archive/'+ str(date)
            page = requests.get(url)    
            result = BeautifulSoup(page.content,'lxml')
            UL = result.find_all('article') 
            t = t+1
        if len(UL)!=0:
            for news in range(len(UL)):
                body1 = UL[news].find_all('h3') #headlines
                if len(body1)!=0:
                    body2 = body1[0].find_all('a')[0].text #headlines
                    body3 = body2.strip().split('\n ') #headlines
                else:
                    body3 = ['']
                bodyab1 = UL[news].find_all('p') #abstract        
                if len(bodyab1):
                    bodyab2 = bodyab1[0].text #abstract
                    bodyab3 = bodyab2.strip().split('\n ') #abstract
                else:
                    bodyab3=['']
                Headlines_item = body3[0]
                Abstracts_item = bodyab3[0]
                Headlines_Day.append(Headlines_item)
                Abstracts_Day.append(Abstracts_item)
            everyday_results = [date, Headlines_Day, Abstracts_Day]
            All_News.append(everyday_results)
        else:
            everyday_results = [date,[],[]]
            All_News.append(everyday_results)
          
    return All_News


In [78]:
output_news = news(date_loop_list)

In [79]:
len(output_news)

526

In [80]:
# Sample of headlines
output_news[10][1][0:20]

['Falling for Pharrell',
 'Pepper...and Salt',
 'What Will a Teen Do to Earn a Smartphone?',
 'Nursing-Home Executive Backs Ex-Gov. Rowland on Stand',
 'Eli Manning Could Use a Hand or Two',
 'Soccer Team Interested in Queens Site',
 'Flores Powers Mets With Two Homers',
 'Moderate Uighur Scholar Faces Trial',
 'College Building Booms, and City Reaps Benefits',
 'Opposing Reports on Cost of Brooklyn Bridge Park',
 'New Director Brings Fresh Eyes to Dia',
 'Being a Big Baby About a Little Rain',
 'Union Wants Neutral Party to Hear Ray Rice Appeal',
 'Kremlin to Base Full Military Unit in Crimea',
 'Mayors Push for Immigrants to Become Citizens',
 "Credit Suisse's Leveraged Lending Draws Scrutiny From Fed",
 'Brazil Official Recommends Ban on Pro-Rousseff Ad',
 "Argentina Protests U.S. Diplomat's Remarks",
 'New York State, Nuclear Plant Operator Clash Over Fate of Fish',
 'Does Fantasy Baseball Translate to Reality?']

## Convert the web scraping results to dataframe

In [81]:
def news_to_dataframe(output_news):
    news_df = pd.DataFrame()
    date = []
    news_title = []
    news_abstract = []
    for day in output_news:
        date.append(day[0])  # date
        news_title.append(day[1]) # news_title
        news_abstract.append(day[2]) # news_abstract
    news_df['Date'] = date
    news_df['News_title'] = news_title
    news_df['News_abstract'] = news_abstract
    
    return news_df

In [15]:
news_data1 = news_to_dataframe(output_news) 

In [27]:
news_data2 = news_to_dataframe(output_news) 

In [41]:
news_data3 = news_to_dataframe(output_news) 

In [69]:
news_data4 = news_to_dataframe(output_news) 

In [82]:
news_data5 = news_to_dataframe(output_news) 

In [16]:
# 1
## start= datetime(2008,7,1)
## end = datetime(2008,9,1)
df1 = news_data1
df1.shape #(43, 3)

(43, 3)

In [28]:
# 2
## start= datetime(2008,9,1)
## end = datetime(2010,9,1)
df2 = news_data2
df2.shape #(505, 3)

(505, 3)

In [42]:
# 3
## start= datetime(2010,9,1)
## end = datetime(2012,9,1)
df3 = news_data3
df3.shape  #(506, 3)

(506, 3)

In [46]:
df3 = df3.ix[1:]
df3.shape  #(505, 3)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


(505, 3)

In [70]:
# 4
# start= datetime(2012,9,1)
# end = datetime(2014,9,1)
df4 = news_data4
df4.shape 

(500, 3)

In [84]:
# 5
## start= datetime(2014,9,1)
## end = datetime(2016,10,1)
df5 = news_data5
df5.shape 

(526, 3)

In [29]:
df1.isnull().sum()

Date             0
News_title       0
News_abstract    0
dtype: int64

In [30]:
df2.isnull().sum()

Date             0
News_title       0
News_abstract    0
dtype: int64

In [43]:
df3.isnull().sum()

Date             0
News_title       0
News_abstract    0
dtype: int64

In [71]:
df4.isnull().sum()

Date             0
News_title       0
News_abstract    0
dtype: int64

In [85]:
df5.isnull().sum()

Date             0
News_title       0
News_abstract    0
dtype: int64

In [17]:
df1.head()

Date                                         News_title  \
0  20080701  [Pepper &hellip; and Salt, CIT Group, Oncothyr...   
1  20080702  [Pepper &hellip; and Salt, Qimonda, Maxygen: B...   
2  20080703  [Pepper &hellip; and Salt, Flagstar Bancorp, D...   
3  20080707  [Pepper &hellip; and Salt, A Kenyan Safari, SP...   
4  20080708  [Pepper &hellip; and Salt, Office Depot, 1st P...   

                                       News_abstract  
0  [Send reprint requests to, Markets Data Center...  
1  [Send reprint requests to, Markets Data Center...  
2  [Send reprint requests to, Markets Data Center...  
3  [Send reprint requests to, About an hour's fli...  
4  [Send reprint requests to, Markets Data Center...

In [18]:
df1.tail()

Date                                         News_title  \
38  20080825  [Regulators Close Ninth U.S. Bank, Farber as a...   
39  20080826  [Cott, Expedia wts: Biggest Price Decliners (C...   
40  20080827  [Summary Hed, Goldman Sachs, SPDR S&P 500: Mon...   
41  20080828  [Pepper &hellip; and Salt, Apple, iShares Russ...   
42  20080829  [Pepper &hellip; and Salt, PMI Group, Providen...   

                                        News_abstract  
38  [State and federal regulators shut down Columb...  
39  [Markets Data Center: Biggest Price Decliners....  
40  [Libba Galloway, deputy commissioner of the LP...  
41  [Send reprint requests to, Markets Data Center...  
42  [Send reprint requests to, Markets Data Center...

In [31]:
df2.head()

Date                                         News_title  \
0  20080902  [Pepper &hellip; and Salt, Energy Select Secto...   
1  20080903  [Pepper &hellip; and Salt, Construction Sees D...   
2  20080904  [Pepper &hellip; and Salt, Management Author H...   
3  20080905  [Test.2 0905, Test.1 0905, Pepper &hellip; and...   
4  20080908  [Pepper &hellip; and Salt, Fannie Mae, Biodel:...   

                                       News_abstract  
0  [Send reprint requests to, Markets Data Center...  
1  [Send reprint requests to, Construction spendi...  
2  [Send reprint requests to, Michael Hammer, an ...  
3  [SummaryText, SummaryText, Send reprint reques...  
4  [Send reprint requests to, Markets Data Center...

In [32]:
df2.tail()

Date                                         News_title  \
500  20100826  [Corrections & Amplifications, The Toll of the...   
501  20100827  [What's Hot…and Not, Katrina: Five Years Later...   
502  20100830  [Yankees 11, A's 5, Braves 9, Mets 3, Pepper.....   
503  20100831  [Corrections &Amplifications, Noah Educational...   
504  20100901  [Visiting Utah, Pepper...and Salt, Sunrise Sen...   

                                         News_abstract  
500  [Corrections & Amplifications, A look at the c...  
501  [A graphical look at how different investments...  
502  [The Yankees posted a 13-hit night as they pic...  
503  [Corrections &Amplifications, Markets Data Cen...  
504  [Stephanie Simon offers tips on finding the st...

In [47]:
df3.head()

Date                                         News_title  \
1  20100902  [The Fall Season, Arbitrage: American Express ...   
2  20100903  [What's Hot…and Not, Pepper...and Salt, Cascad...   
3  20100907  [Corrections & Amplifications, Pepper...and Sa...   
4  20100908  [Vital Signs, Pepper...and Salt, Grupo Casa Sa...   
5  20100909  [Pepper...and Salt, Spartech, Fuqi Internation...   

                                       News_abstract  
1  [Our critics and reporters on the most anticip...  
2  [A graphical look at how different investments...  
3  [Corrections & Amplifications, Pepper...and Sa...  
4  [Revolving consumer credit outstanding (almost...  
5  [Pepper...and Salt, Markets Data Center: Bigge...

In [45]:
df3.tail()

Date                                         News_title  \
501  20120827  [Pepper...and Salt, What's News: Business & Fi...   
502  20120828  [Pepper...and Salt, CD Yields Hold Steady, Cor...   
503  20120829  [Pepper...and Salt, Dycom Industries, ArQule: ...   
504  20120830  [Corrections & Amplifications, Pepper...and Sa...   
505  20120831  [Sentiment Tracker: Starving to Live, Pepper.....   

                                         News_abstract  
501  [Pepper...and Salt, What's News: Business & Fi...  
502  [Pepper...and Salt, Yields on certificates of ...  
503  [Pepper...and Salt, Markets Data Center: Bigge...  
504  [Corrections & Amplifications for the edition ...  
505  [The online buzz about a new study showing tha...

In [72]:
df4.head()

Date                                         News_title  \
0  20120904  [Pepper...and Salt, CD Yields Hold Steady, Tre...   
1  20120905  [Pepper...and Salt, Smartphone Wars, Dynamic P...   
2  20120906  [Kimberly Ovitz Spring 2013, Pepper...and Salt...   
3  20120907  [2005, Pepper...and Salt, Pepper...and Salt, J...   
4  20120910  [Say What?, Pepper...and Salt, U.S. Sets T-Bil...   

                                       News_abstract  
0  [Pepper...and Salt, Yields on certificates of ...  
1  [Pepper...and Salt, Explore and compare featur...  
2  [Kimberly Ovitz's spring 2013 collection was i...  
3  [The last time Wisconsin played a road game ag...  
4  [A hearing quiz, Pepper...and Salt, The Treasu...

In [73]:
df4.tail()

Date                                         News_title  \
495  20140825  [Pepper...and Salt, Crafty Men Unwind With Kni...   
496  20140826  [Lender Offers Up to $4 Billion Loan For Detro...   
497  20140827  [USC Says Player Admits Lying About Heroic Sto...   
498  20140828  [Street Styles From Asia, Oklahoma Denied Waiv...   
499  20140829  [Pepper...and Salt, What's News: Business & Fi...   

                                         News_abstract  
495  [Pepper...and Salt, Several groups in New York...  
496  [Art Capital Group raised its proposed loan to...  
497  [Southern California said it has suspended def...  
498  [Fashion from the streets of Tokyo, Sydney, Be...  
499  [Pepper...and Salt, What's News: Business & Fi...

In [86]:
df5.head()

Date                                         News_title  \
0  20140902  [EU Mulls Modest Increase in Sanctions, Austra...   
1  20140903  [Kansas Democrat Exits U.S. Senate Race, A Spi...   
2  20140904  [Greater New York Watch: News Digest, Greater ...   
3  20140905  [Pepper...and Salt, World Watch: News Digest, ...   
4  20140908  [Pepper...and Salt, Clinton, Bush Share Laughs...   

                                       News_abstract  
0  [The European Union will make a decision on fu...  
1  [The Democratic candidate for U.S. Senate in K...  
2  [A roundup of news from across the tri-state r...  
3  [Pepper...and Salt, World Watch: News Digest, ...  
4  [Pepper...and Salt, Former Presidents Bill Cli...

In [87]:
df5.tail()

Date                                         News_title  \
521  20160926  [What’s News: World-Wide, Indian Stocks Emerge...   
522  20160927  [Takata Air Bags Recalled in South Korea, ペレス前...   
523  20160928  [Photos of the Day:Asia, Russians Score as Lon...   
524  20160929  [Photos of the Day: Asia, Traders to Scour Jap...   
525  20160930  [Vietnam to Launch New Benchmark Stock Index, ...   

                                         News_abstract  
521  [What’s News: World-Wide, Indian shares are on...  
522  [South Korea’s transport ministry has ordered ...  
523  [A man visits the Forbidden City in Beijing, a...  
524  [Bodybuilders compete over their muscles in a ...  
525  [Vietnam said it will launch VNX Allshare on O...

In [88]:
news_df = [df1,df2,df3,df4,df5]
news_df = pd.concat(news_df)
news_df.shape

(2079, 3)

In [89]:
news_df.head(10)

Date                                         News_title  \
0  20080701  [Pepper &hellip; and Salt, CIT Group, Oncothyr...   
1  20080702  [Pepper &hellip; and Salt, Qimonda, Maxygen: B...   
2  20080703  [Pepper &hellip; and Salt, Flagstar Bancorp, D...   
3  20080707  [Pepper &hellip; and Salt, A Kenyan Safari, SP...   
4  20080708  [Pepper &hellip; and Salt, Office Depot, 1st P...   
5  20080709  [Pepper &hellip; and Salt, Circor Int'l, 1st P...   
6  20080710  [Pepper &hellip; and Salt, Entercom Communicat...   
7  20080711  [Crossword Solution: American Plan, Pepper &he...   
8  20080714  [Pepper &hellip; and Salt, Washington Mutual, ...   
9  20080715  [Pepper &hellip; and Salt, iShares MSCI Emergi...   

                                       News_abstract  
0  [Send reprint requests to, Markets Data Center...  
1  [Send reprint requests to, Markets Data Center...  
2  [Send reprint requests to, Markets Data Center...  
3  [Send reprint requests to, About an hour's fli...  
4  [Send reprint requests to, Markets Data Center...  
5  [Send reprint requests to, Markets Data Center...  
6  [Send reprint requests to, Markets Data Center...  
7  [The crossword puzzle solution for July 5, 200...  
8  [Send reprint requests to, Markets Data Center...  
9  [Send reprint requests to, Markets Data Center...

In [502]:
news_df.head()

Date                                         News_title  \
0  20080701  [Pepper &hellip; and Salt, CIT Group, Oncothyr...   
1  20080702  [Pepper &hellip; and Salt, Qimonda, Maxygen: B...   
2  20080703  [Pepper &hellip; and Salt, Flagstar Bancorp, D...   
3  20080707  [Pepper … and Salt, A Kenyan Safari, SPDR Trus...   
4  20080708  [Pepper &hellip; and Salt, Office Depot, 1st P...   

                                       News_abstract  
0  [Send reprint requests to, Markets Data Center...  
1  [Send reprint requests to, Markets Data Center...  
2  [Send reprint requests to, Markets Data Center...  
3  [Send reprint requests to prestochas@gmail.com...  
4  [Send reprint requests to, Markets Data Center...

In [90]:
news_df1 = news_df
news_df1.head()

Date                                         News_title  \
0  20080701  [Pepper &hellip; and Salt, CIT Group, Oncothyr...   
1  20080702  [Pepper &hellip; and Salt, Qimonda, Maxygen: B...   
2  20080703  [Pepper &hellip; and Salt, Flagstar Bancorp, D...   
3  20080707  [Pepper &hellip; and Salt, A Kenyan Safari, SP...   
4  20080708  [Pepper &hellip; and Salt, Office Depot, 1st P...   

                                       News_abstract  
0  [Send reprint requests to, Markets Data Center...  
1  [Send reprint requests to, Markets Data Center...  
2  [Send reprint requests to, Markets Data Center...  
3  [Send reprint requests to, About an hour's fli...  
4  [Send reprint requests to, Markets Data Center...

In [509]:
news_df.head()

Date                                         News_title  \
0  20080701  [Pepper &hellip; and Salt, CIT Group, Oncothyr...   
1  20080702  [Pepper &hellip; and Salt, Qimonda, Maxygen: B...   
2  20080703  [Pepper &hellip; and Salt, Flagstar Bancorp, D...   
3  20080707  [Pepper … and Salt, A Kenyan Safari, SPDR Trus...   
4  20080708  [Pepper &hellip; and Salt, Office Depot, 1st P...   

                                       News_abstract  
0  [Send reprint requests to, Markets Data Center...  
1  [Send reprint requests to, Markets Data Center...  
2  [Send reprint requests to, Markets Data Center...  
3  [Send reprint requests to prestochas@gmail.com...  
4  [Send reprint requests to, Markets Data Center...

In [91]:
news_df.to_csv("news_data.csv", header = True)